## Notebook cào dữ liệu

In [4]:
def write_to_json_file(data, file_name):
  import json
  json_object = json.dumps(data, ensure_ascii=False)
  with open(file_name, "w", encoding="utf-8") as outfile:
    outfile.write(json_object)

In [5]:
from bs4 import BeautifulSoup
import requests

url = "https://www.petmart.vn/hoi-dap-thu-y-mien-phi"
page = requests.get(url, verify=False)
soup = BeautifulSoup(page.text, 'html.parser')

c:\Users\ASUS\AppData\Local\Programs\Python\Python38-32\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.petmart.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [6]:
def get_comments(): 
  soup.find(class_='comment')
  comments = soup.find_all("li", class_='comment')
  return comments

def classify_comments(comments):
  ask_list = []
  ask_answer_list = []
  for i in comments:
    content = i.find_all(class_='comment-content')
    if len(content) == 1:
      ask_list.append(content)
    else:
      ask_answer_list.append(content)
  return ask_list, ask_answer_list

Tập dữ liệu được thu thập từ mục hỏi đáp thú y của Petmart có 194 mẫu, câu truyện hỏi đáp. Trong đó chỉ có 59 câu truyện có câu trả lời, còn lại 135 mẫu câu hỏi nhưng bị thiếu câu trả lời.

In [7]:
comments = get_comments() # comments 
[ask_list, ask_answer_list] = classify_comments(comments)
len(comments), len(ask_list), len(ask_answer_list)

(197, 138, 59)

Lấy các mẫu cẩu hỏi và định dạng theo kiểu JSON {ask: "lorem srum...."}
Có 135 mẫu kiểu này

In [8]:
# list only have ask but haven't answer yet
ask_data = []
for item in ask_list:
  ask = (item[0].contents)[0].text
  ask_data.append({"ask": ask})
len(ask_data)
ask_data[:3]

[{'ask': 'Shop ơi, mèo em mèo con hơn 1 tháng, em mới đón bé về mấy ngày, thì nay mèo có dấu hiệu đi phân hơi nhão, lỏng có xíu máu thì em nên làm gì ạ?'},
 {'ask': 'Chó nhà em không biết vì sao bắt đầu nhỏ nước dãi và không khép miệng lại được luôn thè lưỡi ra, bé cũng có ăn chút ít nhưng bây giờ miệng bé sưng phù lên, và không ăn được nữa chỉ uống nước, dáng đi bé uể oải, cổ hơi niểng sang bên khác, mắt lúc nào cũng chảy nước.'},
 {'ask': 'Cho mình hỏi, nhà có nuôi 1 con poodle gần 1 tuổi trước để bạn dưới nhà thì bạn ra đằng sau đi vệ sinh, nay bạn ấy leo cầu thang được là cứ leo lên phòng ngủ chơi về đi vệ sinh bừa bãi, mình muốn hỏi cách huấn luyện hay chỗ nào nhận huấn luyện để mình biết vì bé nhỏ nuôi nhưng đi học cả ngày không có thời gian và cũng không biết cách huấn luyện cho bạn ấy đi vệ sinh đúng nơi đúng chỗ. Nhờ bên cửa hàng hỗ trợ tư vấn dùm. Xin cảm ơn!'}]

Định dạng các câu truyện ngắn theo mẫu câu hỏi - câu trả lời. Mỗi câu truyện sẽ có dạng {"ask": "abc", "answer": "xyz"}.
Đồng thời nếu câu truyện dài thì ta lưu vào list và sẽ có định dạng khác

Hiện tại có 50 câu truyện ngắn chỉ bao gồm có 1 câu hỏi - 1 câu trả lời

In [9]:
ask_answer_data = []
story_list = [] # item with len > 2
for item in ask_answer_list:
  # [ask, answer] = item
  if (len(item) > 2):
    story_list.append(item)
  else:
    [ask, answer] = item
    ask = ask.contents[0].text
    answer = answer.contents[0].text
    ask_answer_data.append({"ask": ask, "answer": answer})
len(ask_answer_data), ask_answer_data[:3]

(50,
 [{'ask': 'Dạ cho em hỏi, mèo nhà em được gần 4 năm tuổi rồi ạ. Thì bình thường trong nhà cũng không chú ý việc tiêm vaccine nên là mèo chưa có mũi tiêm nào. Hai ba hôm gần đây, tự nhiên mèo bị tiêu chảy (phân lỏng có màu hồng nhạt), có cho nhịn ăn cỡ 8 tiếng rồi cho ăn ít ít thành từng bữa (đồ ăn hạt của mèo), nhưng mèo vẫn bị tiêu chảy, có nôn ít (1 lần sau 3 ngày bị tiêu chảy), ăn cũng ít lại. Mong các bác sĩ tư vấn ạ, em cảm ơn nhiều ạ.',
   'answer': 'Chào bạn, Pet Mart hỗ trợ tư vấn tới bạn một số thông tin như sau:\n1. Về vấn đề tiêm phòng, mặc dù mèo 4 năm tuổi trưởng thành sức đề kháng tương đối tốt rồi, nhưng không nên chủ quan. Bạn nên cho bé đi tiêm phòng đầy đủ và định kỳ 1 năm 1 lần nhé.\n2. Gần đây bạn thấy mèo phân bất thường, tiêu chảy thì bạn cần xem lại vấn đề ăn của bé. Cụ thể là đồ dùng ăn có vệ sinh sạch sẽ không? Cung cấp nước sạch, không dùng nước máy! Ngoài ra, bạn xem thức ăn hạt cho mèo là loại gì, của hãng nào? Trước đó có dùng loại này không? Nếu là lo

Câu Chuyện dài sẽ có dạng [{"ask": "hỏi"}, {"answer": "đáp"}, {"ask": "hỏi tiếp"}, {"answer": "đáp tiếp"}]

Hiện tại có 9 câu chuyện dài

In [10]:
stories_data = []
for story in story_list:
  i = 0
  story_data = []
  for item in story:
    if i % 2 == 0:
      ask = item.contents[0].text
      story_data.append({"ask": ask})
    else:
      answer = item.contents[0].text
      story_data.append({"answer": answer})
    i += 1
  stories_data.append(story_data)
print(len(stories_data))
stories_data[0]

9


[{'ask': 'Bé cún nhà em không hiểu sao chiều vẫn bình thường nhưng sáng ngủ dậy lại bị sưng 1 bên mặt, mọi khi thì năng động lắm giờ lầm lì, ít hoạt động và không chịu ăn, em tính đưa bé đi khám nhưng nhà lại bảo quan sát 1-2 ngày. Nhờ Pet Mart tư vấn giải pháp được không ạ?'},
 {'answer': 'Chào bạn theo như thông tin bạn miêu tả, thì khả năng bé chó nhà mình đã bị dị ứng 1 cái gì đó khiến cho mặt bị phù nề. Trường hợp này thì Pet Mart không thể tư vấn online cho bạn được, bạn nên đưa bé đi thú y để bác sĩ kiểm tra trực tiếp sẽ đánh giá được chính xác vấn đề của bé. Hoặc bạn có thể tham khảo thêm bài viết này: https://www.petmart.vn/trieu-chung-benh-di-ung-o-cho-va-cach-dieu-tri 13 triệu chứng bệnh dị ứng ở chó và cách điều trị'},
 {'ask': 'Cho em hỏi luôn với ạ. Chuyện là bé mèo nhà em ban đêm ngày hôm kia bỗng dưng nôn 2,3 lần (hôm sau là hết nôn) nhưng sáng ra em thấy tay bé sưng to lên bất thường, theo dõi được 1 ngày thì thấy nó vẫn sưng và còn thêm chảy mủ với khó di chuyển hơn, 

Ghi tất cả các dữ liệu đã cào được ra file

In [11]:
write_to_json_file(ask_data, "petmart_only_ask_data.json")
write_to_json_file(ask_answer_data, "petmart_ask_answer_data.json")
write_to_json_file(stories_data, "petmart_stories_data.json")